In [ ]:
import os
import sys
import time
import pandas as pd

from linkedin_jobs_scraper import LinkedinScraper, events, query, filters
from resume_matcher.matching import resume_job_desc_match

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate

## LinkedIn Scraping

In [ ]:
JOB_RESULTS = []

def on_data(data):
    JOB_RESULTS.append({
        "title": data.title,
        "company": data.company,
        "location": data.location,
        # "description": data.description,
        "link": data.link,
        "date": data.date
    })

def on_error(error):
    print('[ON_ERROR]', error)

def on_end():
    print('[END]')

def scrape_linkedin_jobs(search_term, location = 'Remote', num_jobs = 10):
    global JOB_RESULTS
    scraper = LinkedinScraper(
        chrome_executable_path=None,
        # chrome_driver_path=None,
        headless=True,
        max_workers=1,
        slow_mo=0.5,
        page_load_timeout=20,
        # browser_user_agent=None,
        # proxy=None
    )
    scraper.on(events.Events.DATA, on_data)
    scraper.on(events.Events.ERROR, on_error)
    scraper.on(events.Events.END, on_end)

    queries = [
        query.Query(
            query=search_term,
            options= query.QueryOptions(
                locations = [location],
                apply_link = True,
                limit = num_jobs,
                filters = query.QueryFilters(
                    experience=[filters.ExperienceLevelFilters.ENTRY_LEVEL],
                    type=[filters.TypeFilters.FULL_TIME]
                ),
            ),
        ),
    ]
    
    scraper.run(queries)
    time.sleep(2)
    return JOB_RESULTS.copy()
    

In [ ]:
JOB_QUERY = "AI Engineer"    # Edit as desired
JOB_LOCATION = "Remote"
NUM_JOBS = 10

scraped_jobs = scrape_linkedin_jobs(JOB_QUERY, location=JOB_LOCATION, num_jobs=NUM_JOBS)

print(f"Scraped {len(scraped_jobs)} jobs.")
print(scraped_jobs)  # Display first 5 jobs